In [4]:
import GLRenderer as GL

In [5]:
renderer = GL.setup_renderer(GL.CameraIntrinsics(), GL.DepthMode(), gl_version=(4,1))

gl_version_for_shaders = "410"
true


GLRenderer.Renderer{GLRenderer.DepthMode}(GLFW.Window(Ptr{Nothing} @0x0000000009542580), GLRenderer.CameraIntrinsics
  width: Int64 640
  height: Int64 480
  fx: Float64 640.0
  fy: Float64 640.0
  cx: Float64 320.0
  cy: Float64 240.0
  near: Float64 0.001
  far: Float64 100.0
, 0x00000003, Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Float32[2.0 0.0 0.0 0.0; 0.0 2.6666667 0.0 0.0; 0.0 0.0 -1.00002 -0.0020000201; 0.0 0.0 -1.0 0.0], 1)

In [6]:
import Revise
import GLRenderer as GL
import Images as I
import MiniGSG as S
import Rotations as R
import PoseComposition: Pose, IDENTITY_POSE, IDENTITY_ORN
import InverseGraphics as T
import NearestNeighbors
import LightGraphs as LG
import Gen
import Random
try
    import MeshCatViz as V
catch
    import MeshCatViz as V    
end
V.setup_visualizer()
import ImageView as IV
import JSON
import Statistics

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/aryan/.julia/packages/MeshCat/oC0sL/src/visualizer.jl:73
Gtk-Message: 10:48:44.368: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.


In [7]:
YCB_DIR = joinpath(dirname(dirname(pathof(T))),"data")
world_scaling_factor = 10.0
id_to_cloud, id_to_shift, id_to_box  = T.load_ycbv_models_adjusted(YCB_DIR, world_scaling_factor);
all_ids = sort(collect(keys(id_to_cloud)));
names = T.load_ycb_model_list(YCB_DIR)

camera = GL.CameraIntrinsics()
renderer = GL.setup_renderer(camera, GL.DepthMode(); gl_version=(3,3))
obj_paths = T.load_ycb_model_obj_file_paths(YCB_DIR)
for id in all_ids
    mesh = GL.get_mesh_data_from_obj_file(obj_paths[id])
    mesh = T.scale_and_shift_mesh(mesh, world_scaling_factor, id_to_shift[id])
    GL.load_object!(renderer, mesh)
end

function place_objects(config, all_obj_ids, rot)
    if rot == 0
        rotation = [0.7,1.05,0.87,0.5,0.95,0.18,0.2,0.16,0.42,0.15,1.2,1.24,0.27,0.4,0.28,1.03,0.6,0.1,0.2,0.15,0.26, 10.0]
    elseif rot == pi/2
        rotation = [0.5,0.82,0.47,0.33,0.34,0.43,0.63,0.5,0.29,0.9,0.7,0.34,0.8,0.45,0.92,0.5,1.0,0.6,0.85,0.83,0.4, 10.0]
    end
    
    zvals = []
    for z in 1:3
        append!(zvals,rotation[all_obj_ids[z]])
    end
    
    if config == "line"
        xvals = [1.5,0.0,-2.5]
        yvals = [0.0,0.0,0.0]
    elseif config == "nonequtriangle"
        xvals = [-1.5,1.5,1.5]
        yvals = [1.0,-1.5,1.5]
    elseif config == "random"
        xvals = [0.5,0.0,-1.0]
        yvals = [1.75,-1.75,0.75]
    end
    
    all_object_poses = [
        Pose([xvals[1], yvals[1], zvals[1]],R.RotX(rot)),
        Pose([xvals[2], yvals[2], zvals[2]],R.RotX(rot)),
        Pose([xvals[3], yvals[3], zvals[3]],R.RotX(rot))
        ]
    all_clouds = hcat([
        GL.move_points_to_frame_b(id_to_cloud[obj_id], pose)
        for (obj_id, pose) in zip(all_obj_ids, all_object_poses)
    ]...)
    V.reset_visualizer()
    V.viz(all_clouds ./ 10.0)
    return all_object_poses
    end

gl_version_for_shaders = "330"
true


place_objects (generic function with 1 method)

In [8]:
# all objects we are using in this experiment
allobjs = [2,4,5,7,10,11,12,13,14,18]

radius = 6.5
z = 6.5
inferred_poses = [
    Pose([sin(ang)*radius, cos(ang)*radius, z], R.RotZ(-ang)*R.RotX(-atan(radius / z))*R.RotY(pi))
    for ang in 0.0:pi/4:2*pi
]

9-element Vector{Pose}:
 Pose⟨pos=[0.0, 6.5, 6.5], orientation=(w=5.657130561438503e-17, x=-2.343260202663149e-17, y=0.9238795325112867, z=-0.3826834323650898)⟩
 Pose⟨pos=[4.596194077712559, 4.59619407771256, 6.5], orientation=(w=0.1464466094067261, x=-0.3535533905932737, y=-0.8535533905932737, z=0.3535533905932737)⟩
 Pose⟨pos=[6.5, 3.9801020972288977e-16, 6.5], orientation=(w=0.27059805007309845, x=-0.6532814824381883, y=-0.6532814824381884, z=0.2705980500730985)⟩
 Pose⟨pos=[4.59619407771256, -4.596194077712559, 6.5], orientation=(w=0.3535533905932737, x=-0.8535533905932737, y=-0.3535533905932739, z=0.1464466094067263)⟩
 Pose⟨pos=[7.960204194457795e-16, -6.5, 6.5], orientation=(w=0.3826834323650898, x=-0.9238795325112867, y=-8.000390764101652e-17, z=8.000390764101652e-17)⟩
 Pose⟨pos=[-4.596194077712559, -4.59619407771256, 6.5], orientation=(w=0.35355339059327373, x=-0.8535533905932737, y=0.3535533905932737, z=-0.1464466094067261)⟩
 Pose⟨pos=[-6.5, -1.1940306291686693e-15, 6.5], orient

In [9]:
# allvox = size of ALL_VOXELS
# occupied, occluded, free
function getJSON(filename)
    open(filename,"r") do f
        return JSON.parse(f)
    end
end

function extractImgFromJSON(filename)
    str = split(filename, '/')
    trip = split(str[3], '_')
    trip = [parse(Int, ss) for ss in trip]
    config = str[4]
    view = split(str[5], '.')[1]
    view = [parse(Int, ss) for ss in view]
    return trip,config,view
end

# function getConsistency(viewpoint1, viewpoint2, epsilon)  
#     vox_likelihoods = zeros(length(viewpoint1[1]))
#     avg = 0
#     for i in 1:length(viewpoint1[1])
#         if ((viewpoint1[1][i] == 1 && viewpoint2[1][i] == 1) || (viewpoint1[3][i] == 1 && viewpoint2[3][i] == 1))
#             vox_likelihoods[i] = 1 - epsilon
#             avg += 1 - epsilon
#         elseif ((viewpoint1[1][i] == 1 && viewpoint2[3][i] == 1) ||
#             (viewpoint1[3][i] == 1 && viewpoint2[1][i] == 1))
#             vox_likelihoods[i] = epsilon
#             avg += epsilon
#         elseif (viewpoint1[2][i] == 1 && viewpoint2[2][i] == 1)
#             vox_likelihoods[i] = 0.25
#             avg += 0.25
#         else
#             vox_likelihoods[i] = 0.5
#             avg += 0.5
#         end
#     end
#     return vox_likelihoods,(avg/length(viewpoint1[1]))
# end


function getConsistency(viewpoint1, viewpoint2, epsilon)  
    vox_likelihoods = zeros(length(viewpoint1[1]))
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for i in 1:length(viewpoint1[1])
        if (viewpoint1[1][i] == 1 && viewpoint2[1][i] == 1) 
            TP += 1
        elseif (viewpoint1[3][i] == 1 && viewpoint2[3][i] == 1)
            TN += 1
        elseif (viewpoint1[1][i] == 1 && viewpoint2[3][i] == 1)
            FN += 1
        elseif (viewpoint2[1][i] == 1 && viewpoint1[3][i] == 1)
            FP += 1
        elseif (viewpoint1[2][i] == 1 && viewpoint2[1][i] == 1) || (viewpoint2[2][i] == 1 && viewpoint1[1][i] == 1)
            TP += 0.5
            FP += 0.5
        elseif (viewpoint1[2][i] == 1 && viewpoint2[3][i] == 1) || (viewpoint2[2][i] == 1 && viewpoint1[3][i] == 1)
            TN += 0.5
            FN += 0.5
        else
            TP += 0.25
            TN += 0.25
            FP += 0.25
            FN += 0.25
        end
    end
    consistancy_score = 2* TP / (2*TP + FP + FN)
    return consistancy_score
end

function getAllVoxels(config,objs,RESOLUTION)
    poses = place_objects(config, objs, 0)
    depth_images = [GL.gl_render(renderer, objs, poses, cam_pose) for cam_pose in inferred_poses];
    clouds = [GL.depth_image_to_point_cloud(d, camera) for d in depth_images];
    pose_adjusted_cloud = hcat([T.move_points_to_frame_b(c,p) for (c,p) in zip(clouds, inferred_poses)]...);
    V.viz(pose_adjusted_cloud ./ 10.0)
    mins,maxs = T.min_max(pose_adjusted_cloud)
    mins,maxs = floor.(Int,mins ./ RESOLUTION) * RESOLUTION , ceil.(Int, maxs ./RESOLUTION) * RESOLUTION
    mins .-= RESOLUTION * 5
    maxs .+= RESOLUTION * 5
    dimensions = [length(collect(mins[1]:RESOLUTION:maxs[1])),
              length(collect(mins[2]:RESOLUTION:maxs[2])),
              length(collect(mins[3]:RESOLUTION:maxs[3]))]
    ALL_VOXELS = hcat([[a,b,c] for a in collect(mins[1]:RESOLUTION:maxs[1])
                           for b in collect(mins[2]:RESOLUTION:maxs[2])
                           for c in collect(mins[3]:RESOLUTION:maxs[3])]...);
    return ALL_VOXELS
end

function setAllVoxels(trip1,trip2,trip3,config1,config2,config3)
    ALL_VOXELS1 = getAllVoxels(config1,trip1,RESOLUTION)
    ALL_VOXELS2 = getAllVoxels(config2,trip2,RESOLUTION)
    ALL_VOXELS3 = getAllVoxels(config3,trip3,RESOLUTION)
    if (length(ALL_VOXELS1[2]) > length(ALL_VOXELS2[2])) && (length(ALL_VOXELS1[2]) > length(ALL_VOXELS3[2]))
        return ALL_VOXELS1
    elseif (length(ALL_VOXELS2[2]) > length(ALL_VOXELS1[2])) && (length(ALL_VOXELS2[2]) > length(ALL_VOXELS3[2]))
        return ALL_VOXELS2
    else
        return ALL_VOXELS3
    end
end

function getVpt(trip,config,view,ALL_VOXELS,RESOLUTION)  
    cam_pose = inferred_poses[view][1] 
    poses = place_objects(config, trip, 0)
    depth_image = GL.gl_render(renderer, trip, poses, cam_pose)
    vpt = T.get_occ_ocl_free(T.get_points_in_frame_b(ALL_VOXELS, cam_pose), camera, depth_image, 2*RESOLUTION)
    return vpt
end

function getAllModelMatches(trialfilename,RESOLUTION)
    matches = Dict()
    responses = Dict()
    trials = getJSON(trialfilename)
    for trial in trials
        s = trial["img_sample"]
        l = trial["img_lure"]
        t = trial["img_target"]
        ltype = trial["lure_type"]
        id = trial["trialid"]
        
        s_trip,s_config,s_view = extractImgFromJSON(s)
        l_trip,l_config,l_view = extractImgFromJSON(l)
        t_trip,t_config,t_view = extractImgFromJSON(t)
            
        ALL_VOXELS = setAllVoxels(s_trip,l_trip,t_trip,s_config,l_config,t_config)
    
        s_vpt = getVpt(s_trip,s_config,s_view,ALL_VOXELS,RESOLUTION)
        l_vpt = getVpt(l_trip,l_config,l_view,ALL_VOXELS,RESOLUTION)
        t_vpt = getVpt(t_trip,t_config,t_view,ALL_VOXELS,RESOLUTION)
    
        sl_score = getConsistency(s_vpt, l_vpt, 0.001)
        st_score = getConsistency(s_vpt, t_vpt, 0.001)
        
        if st_score > sl_score
            responses[id] = 1
        else
            responses[id] = 0
        end
    end
    return responses
end

getAllModelMatches (generic function with 1 method)

In [14]:
RESOLUTION = 0.07
modelresponses = getAllModelMatches("3dp3behavior.json",RESOLUTION)

Dict{Any, Any} with 120 entries:
  "32"  => 1
  "29"  => 1
  "1"   => 1
  "54"  => 1
  "78"  => 1
  "81"  => 1
  "101" => 1
  "2"   => 1
  "105" => 1
  "109" => 1
  "74"  => 1
  "41"  => 1
  "65"  => 1
  "51"  => 1
  "53"  => 1
  "106" => 1
  "119" => 1
  "27"  => 1
  "75"  => 1
  "42"  => 1
  "33"  => 1
  "28"  => 1
  "50"  => 1
  "52"  => 1
  "111" => 1
  ⋮     => ⋮

In [15]:
modelresponses

Dict{Any, Any} with 120 entries:
  "32"  => 1
  "29"  => 1
  "1"   => 1
  "54"  => 1
  "78"  => 1
  "81"  => 1
  "101" => 1
  "2"   => 1
  "105" => 1
  "109" => 1
  "74"  => 1
  "41"  => 1
  "65"  => 1
  "51"  => 1
  "53"  => 1
  "106" => 1
  "119" => 1
  "27"  => 1
  "75"  => 1
  "42"  => 1
  "33"  => 1
  "28"  => 1
  "50"  => 1
  "52"  => 1
  "111" => 1
  ⋮     => ⋮

In [16]:
open("modelresponses_NEW_Dice.json","w") do f
    JSON.print(f, [modelresponses])
end

In [105]:
open("modelmatches.json","w") do f
    JSON.print(f, [modelmatches])
end

In [106]:
check = getJSON("modelmatches.json")
print(length(check[1]))

120